In [1]:
library(Seurat)
library(SeuratData)

── Installed datasets ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── SeuratData v0.2.1 ──

✔ ifnb   3.1.0                                                                                                                          ✔ pbmc3k 3.1.4
✔ panc8  3.0.2                                                                                                                          


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Key ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

✔ Dataset loaded successfully
❯ Dataset built with a newer version of Seurat than installed
❓ Unknown version of Seurat installed




In [4]:
data("panc8")
pancreas.list <- SplitObject(panc8, split.by = "tech")
# pancreas.list <- pancreas.list[c("celseq", "celseq2", "fluidigmc1", "smartseq2")]

In [5]:
for (i in 1:length(pancreas.list)) {
    pancreas.list[[i]] <- NormalizeData(pancreas.list[[i]], verbose = FALSE)
    pancreas.list[[i]] <- FindVariableFeatures(pancreas.list[[i]], selection.method = "vst", nfeatures = 2000,
        verbose = FALSE)
}

In [7]:
reference.list <- pancreas.list[c("celseq", "celseq2", "fluidigmc1", "smartseq2")]
pancreas.anchors <- FindIntegrationAnchors(object.list = reference.list, dims = 1:30, verbose=TRUE)

Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 3499 anchors

Filtering anchors

	Retained 2821 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 2176 anchors

Filtering anchors

	Retained 1841 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 2775 anchors

Filtering anchors

	Retained 2479 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 3516 anchors

Filtering anchors

	Retained 2701 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 6173 anchors

Filtering anchors

	Retained 4635 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 2723 anchors

Filtering anchors

	Retained 1768 anchors



In [15]:
sample.tree=rbind(
    c(-2,-3), # 3->2
    c(1,-1), # 1->{2,3}
    c(2,-4) # 4->{2,3,1}
)
print(sample.tree)
expr.integrated <- IntegrateData(anchorset = pancreas.anchors, dims = 1:30, sample.tree=sample.tree)

     [,1] [,2]
[1,]   -2   -3
[2,]    1   -1
[3,]    2   -4


Merging dataset 3 into 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 1 into 2 3

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 4 into 2 3 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“Adding a command log without an assay associated with it”


In [27]:
sample.tree=rbind(
    c(-1,-2), # 2->1
    c(1,-4), # 4->{1,2}
    c(2,-3) # 3->{1,2,4}
)
print(sample.tree)
expr.integrated <- IntegrateData(anchorset = pancreas.anchors, dims = 1:30, sample.tree=sample.tree, preserve.order=TRUE)
# preserve.order=TRUE guarantees that 2 is merged to 1 and not 1 merged to 2

     [,1] [,2]
[1,]   -1   -2
[2,]    1   -4
[3,]    2   -3


Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 4 into 1 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 3 into 1 2 4

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“Adding a command log without an assay associated with it”


In [26]:
genes=rownames(expr.integrated)

# dataset 1, celseq is changed
print(
    all(
        expr.integrated[genes,expr.integrated$tech=="celseq"][["integrated"]]@data
        ==expr.integrated[genes,expr.integrated$tech=="celseq"][["RNA"]]@data
    )
)

# dataset 2, celseq2 is unchanged
print(
    all(
        expr.integrated[genes,expr.integrated$tech=="celseq2"][["integrated"]]@data
        ==expr.integrated[genes,expr.integrated$tech=="celseq2"][["RNA"]]@data
    )
)

[1] FALSE
[1] TRUE
